In [1]:
from javascript import require, On, Once, AsyncTask, once, off
import math
from actions import Actions

AggregateError [ECONNREFUSED]: 

    at internalConnectMultiple (node:net:1117:18)

    at afterConnectMultiple (node:net:1684:7) {

  code: 'ECONNREFUSED',

  [errors]: [

    Error: connect ECONNREFUSED ::1:3000

        at createConnectionError (node:net:1647:14)

        at afterConnectMultiple (node:net:1677:16) {

      errno: -4078,

      code: 'ECONNREFUSED',

      syscall: 'connect',

      address: '::1',

      port: 3000

    },

    Error: connect ECONNREFUSED 127.0.0.1:3000

        at createConnectionError (node:net:1647:14)

        at afterConnectMultiple (node:net:1677:16) {

      errno: -4078,

      code: 'ECONNREFUSED',

      syscall: 'connect',

      address: '127.0.0.1',

      port: 3000

    }

  ]

}

AggregateError [ECONNREFUSED]: 

    at internalConnectMultiple (node:net:1117:18)

    at afterConnectMultiple (node:net:1684:7) {

  code: 'ECONNREFUSED',

  [errors]: [

    Error: connect ECONNREFUSED ::1:3000

        at createConnectionError (node:net:

In [2]:
# Import the javascript libraries
mineflayer = require("mineflayer")
pathfinder = require('mineflayer-pathfinder')
vec3 = require("vec3")
# Global bot parameters
serverHost = "localhost"
serverPort = 3000
reconnect = True

In [3]:
class Agent:

    def __init__(self, playerUsername, botName, serverHost, serverPort, reconnect=True):
        self.playerUsername = playerUsername
        self.botName = botName
        self.serverHost = serverHost
        self.serverPort = serverPort
        self.reconnect = reconnect
        self.bot = None
        self.botArgs = {
            "host": serverHost,
            "port": serverPort,
            "username": botName,
            "hideErrors": False,
        }
        
        self.createBot()
        

    def createBot(self):
        self.bot = mineflayer.createBot(self.botArgs)
        self.bot.loadPlugin(pathfinder.pathfinder)
        self.mcData = require('minecraft-data')(self.bot.version)
        self.movements = pathfinder.Movements(self.bot, self.mcData)
        self.bot.pathfinder.setMovements(self.movements)
        self.actions = Actions(self.bot, self.mcData, self.movements,self.playerUsername)
        self.startEvents()
        return self.bot
    
    def log(self, message):
        print(f"[{self.bot.username}] {message}")

    def startEvents(self):
        bot = self.bot
        botName = self.botName
        reconnectState = {"reconnect": self.reconnect}

        @On(bot, "login")
        def handleLogin(this):
            botSocket = bot._client.socket
            print(f"[{botName}] Logged in to {botSocket.server if botSocket.server else botSocket._host }")

        @On(bot, "kicked")
        def handleKicked(this, reason, loggedIn):
            if loggedIn:
                print(f"[{botName}] Kicked whilst trying to connect: {reason}")

        @On(bot, "messagestr")
        def handleMessagestr(this, message, messagePosition, jsonMsg, sender, verified=None):
            if messagePosition == "chat" and "quit" in message:
                reconnectState["reconnect"] = False
                this.quit()
            elif messagePosition == "chat" and'come' in message:
                localPlayers = bot.players
                for player in localPlayers:
                        playerData = localPlayers[player]
                        if playerData["uuid"] == sender:
                            target = localPlayers[player].entity
                            
                if not target:
                    bot.chat("I don't see you !")
                    return
                pos = target.position
                bot.pathfinder.setMovements(self.movements)
                bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, 1))
            

        @On(bot, "end")
        def handleEnd(this, reason):
            print(f"[{botName}] Disconnected: {reason}")
            # Turn off old events
            off(bot, "login", handleLogin)
            off(bot, "kicked", handleKicked)
            off(bot, "messagestr", handleMessagestr)

            # Reconnect if the reconnect flag is set to True
            if reconnectState["reconnect"]:
                print(f"[{botName}] Attempting to reconnect")
                self.createBot()

            # Last event listener
            off(bot, "end", handleEnd)
        


  



In [4]:
# agent = Agent('jessica030327',"Johnny", "localhost", 3000)
# agent1 = Agent('jessica030327',"Cassie", "localhost", 3000)
agent2 = Agent('jessica030327',"Jon", "localhost", 3000)


[Jon] Logged in to localhost


In [6]:
chestPosition = vec3(-210,67, 498)
# agent2.actions.moveToChest(chestPosition)
agent2.actions.pathfinding(chestPosition,findPlayer = False)

[Jon] Disconnected: socketClosed
[Jon] Attempting to reconnect


In [10]:
chestPosition = vec3(-212,68, 489)
agent2.actions.placeItem( 'bamboo', chestPosition,"up")


In [ ]:
agent2.bot.inventory.items()

In [ ]:
agent2.bot.inventory.items()
# agent2.equip('dirt','hand')

In [15]:
chestPosition = vec3(-210,67, 489)
agent2.moveToChest(chestPosition)

[Jon] No chest at Vec3 { x: -210, y: 67, z: 489 }, it is air


In [ ]:
goalPosition = vec3(-260, 37, 450)
agent2.pathfinding(goalPosition,findPlayer = True)
# agent1.pathfinding(goalPosition,findPlayer = False)
# agent.pathfinding(goalPosition,findPlayer = False)

In [ ]:
goalPosition.minus(goalPosition)

In [ ]:
# agent2.left(10)
# agent2.right(30)
# agent2.forward(5)
agent2.forward(1)

In [ ]:
mcData = require('minecraft-data')(agent2.bot.version)
mcData.itemsByName

In [ ]:
def log(message):
        print(f"[{agent2.bot.username}] {message}")

def place_item(agent, name, position, direction):
    # Return if name is not string
    if not isinstance(name, str):
        raise ValueError("name for place_item must be a string")
    
    itemByName = mcData.itemsByName[name]
    if not itemByName:
        raise ValueError("No item named {name}")

    item = agent2.bot.inventory.findInventoryItem(itemByName.id)
    # print(item)
    if not item:
        agent2.bot.chat(f"No {name} in inventory")
        return
    else:
        agent2.bot.chat(f"Found {name} in inventory")
        
    itemCount = item.count

    direction_vectors = {
        "up": vec3(0, 1, 0),
        "down": vec3(0, -1, 0),
        "forward": vec3(1, 0, 0),
        "back": vec3(-1, 0, 0),
        "left": vec3(0, 0, 1),
        "right": vec3(0, 0, -1),
    }

    if direction not in direction_vectors:
        raise ValueError(f"Invalid direction {direction}. Choose from 'up', 'down', 'forward', 'back', 'left', 'right'.")
    
    face_vector = direction_vectors[direction]
    reference_block = None
    block_position = position.minus(face_vector)
    block = agent2.bot.blockAt(block_position)
    if block and block.name != "air":
        reference_block = block
        agent2.bot.chat(f"Placing {name} on {block.name}")

    if not reference_block:
        agent2.bot.chat(f"No block to place {name} on. You cannot place a floating block.")
        return


    agent2.pathfinding(position)
        
    agent2.equip(item, "hand")
    agent2.bot.placeBlock(reference_block, face_vector)

    agent2.bot.chat(f"Placed {name}")
    # agent2.bot.save(f"{name}_placed")

chestPosition = vec3(-212,68, 489)

place_item(agent2, 'bamboo', chestPosition,"up")

In [ ]:
agent2.forward(3)

In [ ]:
referenceBlock = agent2.bot.blockAt(vec3(-207, 65, 485))
faceVector = vec3(0, 1, 0)

In [ ]:
referenceBlock

In [ ]:
agent2.bot.placeBlock(referenceBlock, faceVector)

In [ ]:
agent2.bot.inventory.items()